<a href="https://colab.research.google.com/github/zetavg/LLaMA-LoRA-Tuner/blob/main/LLaMA_LoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌾🎛️ Khao-LoRA Tuner

TL;DR: **Runtime > Run All** (`⌘/Ctrl+F9`). Takes about 5 minutes to start. You will be promped to authorize Google Drive access.

In [ ]:
%%html
<audio src="https://github.com/anars/blank-audio/raw/master/1-hour-of-silence.mp3" autoplay muted loop controls />

In [ ]:
%pip install Pillow==9.3.0 numpy==1.23.5

import pkg_resources as r
import PIL
import numpy
for module, min_version in [(PIL, "9.3"), (numpy, "1.23")]:
  lib_version = r.parse_version(module.__version__)
  print(module.__name__, lib_version)
  if lib_version < r.parse_version(min_version):
    raise Exception("Restart the runtime by clicking the 'RESTART RUNTIME' button above (or Runtime > Restart Runtime).")

# Config

In [ ]:
llama_lora_project_url = "https://github.com/krittapat-canik/Khao-tuner.git" # @param {type:"string"}
llama_lora_project_branch = "main" # @param {type:"string"}


In [ ]:
google_drive_folder = "Colab Data/LLaMA-LoRA Tuner" # @param {type:"string"}
google_drive_mount_path = "/content/drive"

from requests import get
from socket import gethostname, gethostbyname
host_ip = gethostbyname(gethostname())
colab_notebook_filename = get(f"http://{host_ip}:9000/api/sessions").json()[0]["name"]
from google.colab import drive
try:
  drive.mount(google_drive_mount_path)

  google_drive_data_directory_relative_path = google_drive_folder
  google_drive_data_directory_path = f"{google_drive_mount_path}/My Drive/{google_drive_data_directory_relative_path}"
  !mkdir -p "{google_drive_data_directory_path}"
  !ln -nsf "{google_drive_data_directory_path}" ./data
  !touch "data/This folder is used by the Colab notebook \"{colab_notebook_filename}\".txt"
  !echo "Data will be stored in Google Drive folder: \"{google_drive_data_directory_relative_path}\", which is mounted under \"{google_drive_data_directory_path}\""
except Exception as e:
  print("Drive won't be mounted!")

In [ ]:
base_model = "decapoda-research/llama-7b-hf" # @param {type:"string"}

# Runtime Info

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

# Prepare the Project

In [ ]:
![ ! -d llama_lora ] && git clone -b {llama_lora_project_branch} --filter=tree:0 {llama_lora_project_url} llama_lora
!cd llama_lora && git add --all && git stash && git fetch origin {llama_lora_project_branch} && git checkout {llama_lora_project_branch} && git reset origin/{llama_lora_project_branch} --hard
![ ! -f llama-lora-requirements-installed ] && cd llama_lora && pip install -r requirements.lock.txt && touch ../llama-lora-requirements-installed

# Launch

In [ ]:
# The following command will launch the app in one shot, but we will not do this here.
# Instead, we will import and run Python code from the runtime, so that each part
# can be reloaded easily in the Colab notebook and provide readable outputs.
# It also resolves the GPU out-of-memory issue on training.
# !python llama_lora/app.py --base_model='{base_model}' --data_dir='./data' --share

In [ ]:
# Set Configs
from llama_lora.llama_lora.config import Config, process_config
from llama_lora.llama_lora.globals import initialize_global
Config.default_base_model_name = base_model
Config.base_model_choices = [base_model]
data_dir_realpath = !realpath ./data
Config.data_dir = data_dir_realpath[0]
Config.load_8bit = True
process_config()
initialize_global()

# Prepare Data Dir
from llama_lora.llama_lora.utils.data import init_data_dir
init_data_dir()

# Load the Base Model
from llama_lora.llama_lora.models import prepare_base_model
prepare_base_model()

## Start Gradio UI 🚀

In [ ]:
import gradio as gr
from llama_lora.llama_lora.ui.main_page import main_page, get_page_title
from llama_lora.llama_lora.ui.css_styles import get_css_styles

with gr.Blocks(title=get_page_title(), css=get_css_styles()) as app:
    main_page()

app.queue(concurrency_count=1).launch(share=True, debug=True, server_name="127.0.0.1")